<a href="https://colab.research.google.com/github/rusonariga/peristaltic-frequency/blob/main/Peristaltic_pump_frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h1 align="center"><i> Peristaltic pump frequency response </i></h1>

Script to read and process pressure input from peristaltic pump, to calculate physical parameters of inlet flow dampener design 

##<h2>Importing libraries</h2>

In [ ]:
# Install needed packages
!pip install detecta

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import io
import os
import matplotlib.pyplot as plt
from detecta import detect_peaks
from google.colab import files
import scipy.fftpack

##<h2>Setting the notebook</h2>

In [ ]:
# Required folder setting
!pwd
%cd content/
if os.path.exists('data'):
  if os.path.exists('results'):
    if os.path.exists('figures'):
      print("Set up ready to upload files")
    else:
      !mkdir figures
  else:
    !mkdir results figures
else:
  !mkdir data results figures

/content
[Errno 2] No such file or directory: 'content/'
/content


In [ ]:
# Upload files
%cd data
files.upload()
%cd ..
!pwd

/content/data


Saving pressure_drop_dataset.xlsx to pressure_drop_dataset.xlsx
/content
/content


#<h1> Functions</h1>

In [ ]:
# ************************* DATA READ  *****************************************
def readData(path="/content/figures", filename=None):
  # Manual upload file and read
  folder = "/content/data/"
  file = "pressure_drop_dataset.xlsx"

  data = pd.read_excel(folder+file)
  data = data.set_index("time")

  ax = data.plot()
  ax.set_title('Static pressure ')
  ax.legend(loc='center right',bbox_to_anchor=(1.3,0.5))
  ax.set_xlabel("Time [s]")
  ax.set_ylabel("Pressure [bar]")
  if filename != None:
    plt.savefig(f"{path}/{filename}.png", dpi=300)
  plt.show()
  return data

# ******************** Detect peaks and valleys ********************************
def detectPeak(values, show=False):
  # Detect all ALL PEAKS and plot data
  peaks = detect_peaks(values, show=show)
  #valleys= detect_peaks(data.to_numpy()[...,idx], mph=0.9, mpd=0.5, valley=True, show=True)

  # Detect all ALL VALLEYS and plot data
  valleys= detect_peaks(values,  mpd=6, valley=True, show=show)

  peak_avg = np.average(values[peaks])
  valley_avg = np.average(values[valleys])
  
  return peak_avg, valley_avg, peaks, valleys

# ******************** PLOT PEAKS AND VALLEYS ********************************
def plotPeak(t, values, peak_avg, valley_avg, peaks, valleys, path="/content/figures", filename=None):
  filename = filename.split(" ")[0]
  # Plot of average, max and mins
  plt.plot(t,values,label= dataset)
  plt.scatter(t[peaks],values[peaks])
  plt.scatter(t[valleys],values[valleys])
  plt.plot(t,t*0+peak_avg, label='Mean peaks', linestyle='--')
  plt.plot(t,t*0+valley_avg, label='Mean valleys', linestyle='--')
  # plt.legend(loc='center right',bbox_to_anchor=(1.35,0.5))
  plt.legend(loc='center right')
  plt.xlabel("Time [s]")
  plt.ylabel("Pressure [bar]")
  plt.title(f"Peaks detection: {dataset}")
  if filename != None:
    plt.savefig(f"{path}/{filename}.png", dpi=300)
  plt.show()

  return None

# *********************** Frequency calculation ********************************
def getFrequency(t, values, path="/content/figures", filename=None):
  filename = filename.split(" ")[0]
  # Get the time values
  x = t
  # Get the pressure values for selected flowrate
  y = values
  # We remove the zero frequency value before performing the fourier transfor to get only the frequencies that introduce variability with the time.
  y = y-y.mean()
  # Size of sampling
  N = x.size
  # Sampling period
  T = x[-1]/N

  # Fourier trasnform calculation
  yf = scipy.fftpack.fft(y)

  # Change time values into frequency space
  xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

  # Maximum frequency calculation
  freq_max = xf[np.argmax(2.0/N * np.abs(yf[:N//2]))]
  
  # Maximum amplitude calculation
  amp_max = yf[np.argmax(2.0/N * np.abs(yf[:N//2]))]

  # Plot for frequencies
  fig, ax = plt.subplots()
  ax.plot(xf, 2.0/N * np.abs(yf[:N//2]))
  ax.plot(freq_max, np.max(2.0/N * np.abs(yf[:N//2])), "C3o", label=f"Max. Frequency {freq_max:0.4f} Hz")
  plt.xlabel("Frequency [Hz]")
  plt.ylabel("Amplitude")
  plt.title(f"Frequencies: {dataset}")
  plt.legend()
  if filename != None:
    plt.savefig(f"{path}/fft-{filename}.png", dpi=300)
  plt.show()

  # Filter frequencies smaller than 50% of the original
  yf[np.abs(yf)<np.abs(yf).max()*50/100] = 0

  # Fourier's transform inversion
  y = scipy.fftpack.ifft(yf)

  # Average is added again
  y = y +data[dataset].values.mean()

  return np.real(y), freq_max, amp_max

# ************************* FINAL PLOT  ***************************************
def plotFinal(t, values, freq_max, y, dataset, path="/content/figures", filename=None):
  filename = filename.split(" ")[0]
  # Plot final curves
  x = t
  plt.plot(t,values, label= dataset)
  plt.plot(x,y, label=f"f = {freq_max:0.4f} Hz\nT = {1/freq_max:0.4f} s")
  plt.legend(loc='center right')
  plt.xlim(1,1+10/freq_max)
  plt.xlabel("Time [s]")
  plt.ylabel("Pressure [bar]")
  plt.title(f"Base frequency fit: {dataset}")
  if filename != None:
    plt.savefig(f"{path}/final-{filename}.png", dpi=300)
  plt.show()
  return None

In [ ]:
amplitude

(2.9111920938162625+19.687371896769246j)

#<h1>Main</h1>

In [ ]:
data = readData()

for dataset in data:
  values = data[dataset].values
  # Find index of desired dataset into data frame
  peak_avg, valley_avg, peaks, valleys = detectPeak(values, show=False)
  # Plot peaks
  plotPeak(data.index.values, values, peak_avg, valley_avg, peaks, valleys, filename=dataset)
  # Get fourier transform and frequency
  y, freq_max, amp_max = getFrequency(data.index.values, values, filename=dataset)
  # Plot final curve
  plotFinal(data.index.values, values, freq_max, y, dataset, filename=dataset)

  # Create dataframe of results
  results_columns=["TIME [S]","MEASURED PRESSURE [bar]","CALCULATED PRESSURE [bar]",
                  "FLOWRATE [ml/min]","MAX FREQUENCY [Hz]","PERIOD [s]",
                   "MAX AMPLITUDE [bar]"]
  results = pd.DataFrame(np.nan, index = np.arange(len(data.index.values)), columns=results_columns)

  results["TIME [S]"] = data.index
  results["MEASURED PRESSURE [bar]"] = data[dataset]
  results["CALCULATED PRESSURE [bar]"] = y
  results["FLOWRATE [ml/min]"][0] = dataset.split(" ")[0]           
  results["MAX FREQUENCY [Hz]"][0] = freq_max
  results["PERIOD [s]"][0] = 1/freq_max
  results["MAX AMPLITUDE [bar]"][0] = amp_max

  # Saving results into CSV
  %cd results/
  results.to_csv(dataset.split(" ")[0]+".csv")
  %cd ..

# <h1>Clearing folders</h1>

In [ ]:
%rm -r data/*
%rm -r figures/*
%rm -r results/*

In [ ]:
!pwd

In [ ]:
%reset

teste teste